# **Deze code pas draaien na het filteren van je dataset**

In [1]:
import webbrowser
import os
import pandas as pd
import json

# # Pad naar de map met JSON-bestanden
# directory = "D:/SSD-Schijf/SCHOOL_PRJ/Sem_6/Code/scraping/batches_2025-03-26_00-38-30"

# # Lijst voor alle observaties
# all_observations = []

# # Doorloop alle JSON-bestanden in de map
# for filename in os.listdir(directory):
#     if filename.endswith(".json"):
#         file_path = os.path.join(directory, filename)
        
#         # JSON-bestand inlezen
#         with open(file_path, "r", encoding="utf-8") as f:
#             data = json.load(f)
        
#         # Voeg observaties toe aan de lijst
#         for obs in data:
#             observation = {
#                 "id": obs["id"],
#                 "date": obs["date"],
#                 "species_name": obs["species_detail"]["name"],
#                 "species_scientific_name": obs["species_detail"]["scientific_name"],
#                 "species": obs["species"],
#                 "number": obs["number"],
#                 "sex": obs["sex"],
#                 "coordinates": obs["point"]["coordinates"] if obs["point"] else None,  # Coördinaten
#                 "life_stage": obs["life_stage"],
#                 "rarity": obs["rarity"],
#                 "validation_status": obs["validation_status"],
#                 "location_name": obs["location_detail"]["name"],
#                 "location_country_code": obs["location_detail"]["country_code"],
#                 "observer_name": obs["user_detail"]["name"],
#                 "permalink": obs["location_detail"]["permalink"]
#             }
#             all_observations.append(observation)

# # Zet alle observaties om in een DataFrame
# df = pd.DataFrame(all_observations)

# df = pd.DataFrame(data)

df = pd.read_csv('observations_verlopige_data_1.csv')

In [2]:
df.head()

,id,species_name,plant_type,species_scientific_name,life_stage,rarity,Species_status,sex,location_name,location_country_code,coordinates,validation_status,permalink
0,310928039,Hybrid Waterlilies,Waterplant,Nymphaea marliacea,35,2,9,U,Rotterdam - Blijdorp (wijk),NL,"[4.4481057, 51.9321591]",N,https://waarneming.nl/locations/44395/
1,327987376,Hybrid Waterlilies,Waterplant,Nymphaea marliacea,1096,2,9,U,Rotterdam - Nieuwe Instituut,NL,"[4.471221, 51.914458]",O,https://waarneming.nl/locations/688793/
2,315921610,Hybrid Waterlilies,Waterplant,Nymphaea marliacea,1096,2,9,U,Rotterdam - Historische Tuin Schoonoord,NL,"[4.473483, 51.908913]",J,https://waarneming.nl/locations/693005/
3,315126729,Hybrid Waterlilies,Waterplant,Nymphaea marliacea,35,2,9,U,Rotterdam - Eiland van Brienenoord,NL,"[4.531772006307293, 51.89874267578125]",O,https://waarneming.nl/locations/676119/
4,327577305,Hybrid Waterlilies,Waterplant,Nymphaea marliacea,35,2,9,U,Rotterdam - Nieuwe Instituut,NL,"[4.471621990203857, 51.914573669433594]",A,https://waarneming.nl/locations/688793/


In [3]:
df["plant_type"].value_counts()

plant_type
Overige plant    24049
Bloem             4215
Gras              1759
Boom              1478
Fruit             1194
Groente           1099
Varen              846
Struik             792
Kruid              615
Waterplant         275
Name: count, dtype: int64

In [4]:
df['permalink']

0         https://waarneming.nl/locations/44395/
1        https://waarneming.nl/locations/688793/
2        https://waarneming.nl/locations/693005/
3        https://waarneming.nl/locations/676119/
4        https://waarneming.nl/locations/688793/
                          ...                   
36317      https://waarneming.nl/locations/9165/
36318     https://waarneming.nl/locations/22688/
36319      https://waarneming.nl/locations/9664/
36320     https://waarneming.nl/locations/18084/
36321     https://waarneming.nl/locations/18084/
Name: permalink, Length: 36322, dtype: object

In [5]:
import requests
import pandas as pd
import base64
import time
import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse, parse_qs, urlencode, urlunparse

# Basis map voor CSV bestanden
BASE_OUTPUT_FOLDER = 'image_datasets'
BATCH_SIZE = 1000  # Aantal afbeeldingen per batch

def create_unique_folder(base_folder):
    """Maak een unieke map aan voor de dataset"""
    if not os.path.exists(base_folder):
        os.makedirs(base_folder)
    return base_folder

def add_filter_to_url(url, filter_params):
    """Voeg filterparameters toe aan een URL"""
    parsed = urlparse(url)
    query_dict = parse_qs(parsed.query)
    
    for key, value in filter_params.items():
        query_dict[key] = [value]
    
    new_query = urlencode(query_dict, doseq=True)
    return urlunparse(parsed._replace(query=new_query))

def fetch_photos_page(url, location_name, images_collected, batch_counter):
    """
    Haalt foto's op en slaat ze op in batches
    """
    filtered_url = add_filter_to_url(url, {'species_group': '10'})
    response = requests.get(filtered_url)
    if response.status_code != 200:
        print(f"Fout bij het ophalen van {filtered_url}: {response.status_code}")
        return images_collected, batch_counter
    
    soup = BeautifulSoup(response.text, 'html.parser')
    photo_link = soup.find('a', href=lambda href: href and '/locations/' in href and '/photos/' in href)
    
    if photo_link:
        base_photo_page_url = urljoin("https://waarneming.nl", photo_link['href'])
        base_photo_page_url = add_filter_to_url(base_photo_page_url, {'species_group': '10'})
        
        all_pages = get_all_pages(base_photo_page_url)
        
        for page_url in all_pages:
            images_collected, batch_counter = fetch_image_data(page_url, location_name, images_collected, batch_counter)
            if len(images_collected) >= BATCH_SIZE:
                batch_counter = save_batch(images_collected, batch_counter)
                images_collected = []  # Reset de verzamelde afbeeldingen
    
    return images_collected, batch_counter

def get_all_pages(photo_page_url):
    """Haalt alle paginanummers op"""
    response = requests.get(photo_page_url)
    if response.status_code != 200:
        print(f"Fout bij het ophalen van {photo_page_url}: {response.status_code}")
        return [photo_page_url]
    
    soup = BeautifulSoup(response.text, 'html.parser')
    pagination_links = soup.find_all('a', href=lambda href: href and 'page=' in href)
    page_urls = set()
    
    for link in pagination_links:
        page_url = urljoin("https://waarneming.nl", link['href'])
        page_url = add_filter_to_url(page_url, {'species_group': '10'})
        page_urls.add(page_url)
    
    page_urls.add(photo_page_url)
    return sorted(page_urls)

def fetch_image_data(photo_page_url, location_name, images_collected, batch_counter):
    """Haalt afbeeldingsdata op"""
    response = requests.get(photo_page_url)
    if response.status_code != 200:
        print(f"Fout bij het ophalen van {photo_page_url}: {response.status_code}")
        return images_collected, batch_counter
    
    soup = BeautifulSoup(response.text, 'html.parser')
    figures = soup.find_all('figure', class_='lightbox-gallery')
    
    for figure in figures:
        species_name = figure.find('span', class_='species-common-name')
        scientific_name = figure.find('i', class_='species-scientific-name')
        image_tag = figure.find('img')
        
        if species_name and scientific_name and image_tag:
            image_url = urljoin("https://waarneming.nl", image_tag['src'].split('?')[0])
            image_name = f"{location_name}_{species_name.text.strip().replace(' ', '_')}_{scientific_name.text.strip().replace(' ', '_')}"
            
            encoded_image = download_and_encode_image(image_url)
            if encoded_image:
                images_collected.append({
                    'Name': image_name,
                    'encoded_pic': encoded_image
                })
                print(f"Afbeelding {len(images_collected)} (Batch {batch_counter}) verwerkt: {image_name}")
                
                # Sla batch op als we de batch size bereiken
                if len(images_collected) >= BATCH_SIZE:
                    batch_counter = save_batch(images_collected, batch_counter)
                    images_collected = []  # Reset de verzamelde afbeeldingen
    
    return images_collected, batch_counter

def download_and_encode_image(image_url):
    """Download en encode afbeelding als base64"""
    try:
        response = requests.get(image_url, timeout=10)
        if response.status_code == 200:
            return base64.b64encode(response.content).decode('utf-8')
        else:
            print(f"Fout bij downloaden: {image_url}")
            return None
    except Exception as e:
        print(f"Fout bij verwerken afbeelding {image_url}: {str(e)}")
        return None

def save_batch(images_collected, batch_counter):
    """Sla een batch afbeeldingen op als CSV"""
    output_folder = create_unique_folder(BASE_OUTPUT_FOLDER)
    csv_path = os.path.join(output_folder, f'images_batch_{batch_counter}.csv')
    
    images_df = pd.DataFrame(images_collected)
    images_df.to_csv(csv_path, index=False)
    print(f"\nBatch {batch_counter} opgeslagen in: {csv_path}")
    print(f"Aantal afbeeldingen in deze batch: {len(images_df)}")
    
    return batch_counter + 1

def process_all_locations(df):
    """Verwerk alle locaties en sla batches op"""
    images_collected = []
    batch_counter = 1
    total_locations = len(df)
    
    print(f"Start verzamelen van afbeeldingen in batches van {BATCH_SIZE}...")
    
    for index, row in df.iterrows():
        location_name = row['name'] if 'name' in df.columns else f"locatie_{index}"
        print(f"\n[{index+1}/{total_locations}] Verwerken: {location_name} (Totaal: {len(images_collected)} in huidige batch)")
        
        images_collected, batch_counter = fetch_photos_page(row['permalink'], location_name, images_collected, batch_counter)
        time.sleep(1)
    
    # Sla de laatste batch op als er nog afbeeldingen zijn
    if len(images_collected) > 0:
        save_batch(images_collected, batch_counter)
    
    print("\nScraping voltooid!")

# Maak de basis map aan als deze niet bestaat
if not os.path.exists(BASE_OUTPUT_FOLDER):
    os.makedirs(BASE_OUTPUT_FOLDER)

# Start het proces
process_all_locations(df)

Start verzamelen van afbeeldingen in batches van 1000...

[1/36322] Verwerken: locatie_0 (Totaal: 0 in huidige batch)
Afbeelding 1 (Batch 1) verwerkt: locatie_0_Mahonie_Berberis_aquifolium
Afbeelding 2 (Batch 1) verwerkt: locatie_0_Tuinjudaspenning_Lunaria_annua
Afbeelding 3 (Batch 1) verwerkt: locatie_0_Groot_hoefblad_Petasites_hybridus
Afbeelding 4 (Batch 1) verwerkt: locatie_0_Sneeuwbal_spec._Viburnum_spec.
Afbeelding 5 (Batch 1) verwerkt: locatie_0_Mahonie_Berberis_aquifolium
Afbeelding 6 (Batch 1) verwerkt: locatie_0_Mahonie_Berberis_aquifolium
Afbeelding 7 (Batch 1) verwerkt: locatie_0_Gewoon_speenkruid_Ficaria_verna
Afbeelding 8 (Batch 1) verwerkt: locatie_0_Gele_kornoelje_Cornus_mas
Afbeelding 9 (Batch 1) verwerkt: locatie_0_Rode_ribes_Ribes_sanguineum
Afbeelding 10 (Batch 1) verwerkt: locatie_0_Grote_ereprijs_Veronica_persica
Afbeelding 11 (Batch 1) verwerkt: locatie_0_Vroegeling_Draba_verna
Afbeelding 12 (Batch 1) verwerkt: locatie_0_Vingerhelmbloem_Corydalis_solida
Afbeeldin

KeyboardInterrupt: 

Ik heb voor nu 37 locaties